In [18]:
%reload_ext autoreload
%autoreload 2
%run ../src/path.py


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Data Poza de Corte

In [62]:
pcorte = pd.read_excel(RAW_DATA_DIR / 'Datos.xlsx', sheet_name='P.corte')
pcorte = pcorte[pcorte.Fecha >= '2022-01-01']
pcorte = pcorte[['Fecha','NaNO3','K', 'MgL','Nivel Sol. Libre cm','Flujo salida', 'Ley bandeja %NaNO3']]

In [67]:
pcorte.isnull().sum()

Fecha                   0
NaNO3                   0
K                       0
MgL                     0
Nivel Sol. Libre cm    38
Flujo salida            0
Ley bandeja %NaNO3      0
dtype: int64

In [64]:
pcorte = pcorte[~pcorte['Ley bandeja %NaNO3'].isna()]

In [30]:
# for index in pcorte[pcorte['Flujo salida'].isnull()].index[8:]:
#     print(pcorte.loc[[index-2, index-1, index, index+1, index+2]])
#     print('\n')

In [66]:
pcorte['Flujo salida'] = pcorte['Flujo salida'].interpolate()

# for index in pcorte[pcorte['Flujo salida'].isnull()].index[8:]:
#     print(pcorte['Flujo salida'].interpolate().loc[[index-2, index-1, index, index+1, index+2]])
#     print('\n')

In [32]:
# sns.scatterplot(pcorte, x='Flujo salida', y='Ley bandeja %NaNO3')

In [33]:
# sns.scatterplot(pcorte, x='NaNO3', y='Ley bandeja %NaNO3')

### Data Temperaturas

In [68]:
temp = pd.read_excel(RAW_DATA_DIR / 'Datos.xlsx', sheet_name='T° P Corte')
temp.head(2)

,Tiempo,Temperatura,Semana,dia,mes
0,2022-01-01 10:01:00,23.5,1,1,1
1,2022-01-01 10:16:00,25.5,1,1,1


In [69]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71789 entries, 0 to 71788
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Tiempo       71789 non-null  datetime64[ns]
 1   Temperatura  71789 non-null  float64       
 2   Semana       71789 non-null  int64         
 3   dia          71789 non-null  int64         
 4   mes          71789 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 2.7 MB


In [70]:
temp_dia = temp.set_index('Tiempo').resample('D').mean()
temp_dia = temp_dia.reset_index()

### Punto Saturación Estimada

In [71]:
pto_sat = pd.read_excel(RAW_DATA_DIR / 'Datos.xlsx', sheet_name='pto sat')
pto_sat.columns =  pto_sat.loc[1]
pto_sat = pto_sat.iloc[2:]
pto_sat = pto_sat[['Fechas','Sat calculada','Sat semanal']]
pto_sat['Fechas'] = pd.to_datetime(pto_sat['Fechas'])
pto_sat[['Sat calculada','Sat semanal']] = pto_sat[['Sat calculada','Sat semanal']].astype(float)
pto_sat = pto_sat[pto_sat['Fechas'] <= '2024-02-18 ']
pto_sat['Sat calculada'] = pto_sat['Sat calculada'].interpolate()
pto_sat.tail()

1,Fechas,Sat calculada,Sat semanal
1505,2024-02-14,443.0,458.0
1506,2024-02-15,438.0,458.0
1507,2024-02-16,433.0,458.0
1508,2024-02-17,428.0,458.0
1509,2024-02-18,423.0,423.0


### Unión Data

In [72]:
data_total = pd.merge(pcorte, temp_dia, left_on='Fecha', right_on='Tiempo')
data_total = data_total.drop('Tiempo', axis=1)

data_total = pd.merge(data_total, pto_sat, left_on='Fecha', right_on='Fechas')
data_total = data_total.drop('Fechas', axis=1)
data_total

,Fecha,NaNO3,K,MgL,Nivel Sol. Libre cm,Flujo salida,Ley bandeja %NaNO3,Temperatura,Semana,dia,mes,Sat calculada,Sat semanal
0,2022-01-01,432.0,50.0,20.070085,NaN,6574.0,3.0,29.866071,1.0,1.0,1.0,448.000000,448.0
1,2022-01-02,430.0,50.0,19.984615,NaN,6579.0,4.9,28.098958,2.0,2.0,1.0,449.333333,448.0
2,2022-01-03,432.0,51.0,19.984615,NaN,6575.0,4.9,28.567708,2.0,3.0,1.0,450.666667,448.0
3,2022-01-04,427.0,51.0,20.442735,NaN,6571.0,4.9,28.072917,2.0,4.0,1.0,452.000000,452.0
4,2022-01-05,407.0,0.0,22.013675,NaN,6557.0,3.9,27.963542,2.0,5.0,1.0,452.161290,452.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,2024-02-03,446.0,42.0,27.324786,64.7,6983.0,26.6,29.458333,5.0,3.0,2.0,424.000000,424.0
764,2024-02-04,452.0,45.0,27.153846,59.5,6835.0,26.6,31.509615,6.0,4.0,2.0,428.250000,424.0
765,2024-02-05,452.0,44.0,26.982906,60.4,6944.0,26.6,31.843750,6.0,5.0,2.0,432.500000,424.0
766,2024-02-06,450.0,43.0,27.153846,NaN,6954.0,0.0,31.864583,6.0,6.0,2.0,436.750000,424.0


In [73]:
null_indexs = data_total[data_total.isna().any(axis=1)].index
null_indexs

Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
        28,  29,  30,  31,  93,  94,  95,  96,  97,  98,  99, 128, 129, 130,
       131, 132, 133, 134, 170, 171, 172, 173, 174, 175, 184, 185, 186, 187,
       188, 189, 228, 230, 232, 240, 241, 242, 243, 244, 245, 253, 350, 513,
       766, 767],
      dtype='int64')

In [74]:
# for i in null_indexs:
#     print(data_total.loc[[i-1, i, i+1, i+2]]['Temperatura'])

In [75]:
data_total['Temperatura'] = data_total['Temperatura'].interpolate()
# data_total.iloc[null_indexs]

In [76]:
data_total['dia'] = data_total['Fecha'].apply(lambda x: x.day)
data_total['mes'] = data_total['Fecha'].apply(lambda x: x.month)
data_total['Semana'] = data_total['Fecha'].apply(lambda x: x.week)
data_total.head(2)

,Fecha,NaNO3,K,MgL,Nivel Sol. Libre cm,Flujo salida,Ley bandeja %NaNO3,Temperatura,Semana,dia,mes,Sat calculada,Sat semanal
0,2022-01-01,432.0,50.0,20.070085,NaN,6574.0,3.0,29.866071,52,1,1,448.000000,448.0
1,2022-01-02,430.0,50.0,19.984615,NaN,6579.0,4.9,28.098958,52,2,1,449.333333,448.0


In [77]:
data_total.to_excel(TRANSFORMED_DATA_DIR / 'data_transformed.xlsx', index=False)